In [1]:
import operator
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from pso_utils import *

from deap import base, creator, gp, tools, algorithms

In [12]:
def target_function(x: np.ndarray, y: np.ndarray):
    return np.add(np.exp(x), np.power(-y, 3))

# Basic example

In [13]:
def target(x, y):
    return np.sin(3*x) - np.sin(3*y)

In [ ]:
# real primitives

In [14]:
pset = gp.PrimitiveSet("MAIN", 2)
pset.addPrimitive(np.add, 2)
pset.addPrimitive(np.negative, 1)
pset.addPrimitive(np.subtract, 2)
pset.addPrimitive(np.multiply, 2)
pset.addPrimitive(np.sin, 1)
pset.addPrimitive(np.cos, 1)
pset.addPrimitive(np.sqrt, 1)

pset.addTerminal(3)
pset.renameArguments(ARG0="x")
pset.renameArguments(ARG1="y")

In [15]:
expr = gp.genFull(pset, min_=1, max_=3)
tree = gp.PrimitiveTree(expr)

In [16]:
tree

In [17]:
# generation of tree individuals

creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMin, pset=pset)

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [18]:
# register generation function in a toolbox

toolbox = base.Toolbox()
toolbox.register('expr', gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('compile', gp.compile, pset=pset)

In [19]:
# visualize tree with str() or compile the function

expr = gp.genFull(pset, min_=1, max_=3)

tree = gp.PrimitiveTree(expr)
print(str(tree))
func = gp.compile(tree, pset)

sin(3)


In [20]:
def evalSymbReg(individual, points):
    # trasform tree expression in a callable function
    func = toolbox.compile(expr=individual)

    # evaluate the mean squared error
    sqerrors = ((func(x) - target(x))**2 for x in points)
    return math.fsum(sqerrors) / len(points)

In [21]:
toolbox.register('evaluate', evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [22]:
def GPrun():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

In [23]:
def target(x):
    return 12

In [ ]:
#    This file is part of EAP.
#
#    EAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    EAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with EAP. If not, see <http://www.gnu.org/licenses/>.

import operator
import math
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
# insert np random uniform
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addPrimitive(random.uniform, 2)
pset.addEphemeralConstant("rand101", partial(random.randint, -2, 2))
pset.addEphemeralConstant("rand102", partial(random.randint, -2, 2))
pset.addTerminal(math.pi/2)
pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) -  target(x))**2 for x in points) # x**4 - x**3 - x**2 - x -
    return math.fsum(sqerrors) / len(points) + 0.7*len(individual),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3) # selction
toolbox.register("mate", gp.cxOnePoint) # crossover
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 100, stats=mstats,
                                   halloffame=hof, verbose=False)
    # print log
    return pop, log, hof

if __name__ == "__main__":
    pop, log, hof = main()

print(log)

   	      	                              fitness                              	                      size                     
   	      	-------------------------------------------------------------------	-----------------------------------------------
gen	nevals	avg        	gen	max        	min    	nevals	std        	avg    	gen	max	min	nevals	std    
0  	300   	1.1913e+30 	0  	3.57391e+32	40.4431	300   	2.05995e+31	3.75333	0  	7  	2  	300   	1.55964
1  	156   	123.073    	1  	229.2      	29.9483	156   	26.0457    	3.68   	1  	11 	1  	156   	1.57404
2  	171   	114.119    	2  	248.729    	12.8536	171   	34.3199    	4.16667	2  	11 	1  	171   	1.80524
3  	188   	102.273    	3  	247.842    	12.8536	188   	33.4046    	4.53   	3  	15 	1  	188   	2.06133
4  	180   	93.3697    	4  	258.1      	9.18272	180   	38.0594    	4.62   	4  	15 	1  	180   	2.18226
5  	143   	72.7703    	5  	197.4      	8.32218	143   	27.229     	5.35   	5  	15 	1  	143   	2.84854
6  	179   	69.7391    	6  	349.116    	

In [26]:
print(f'best solution = {str(hof[0])}')
print(f'best solution size = {len(hof[0])}')
# tree representation

func = gp.compile(hof[0], pset)

best solution = mul(add(1, 2), sub(2, -2))
best solution size = 7


In [27]:
hof[0]

In [ ]:
len(hof[0])

7

In [ ]:
pset.terminals

defaultdict(list,
            {object: [<deap.gp.Terminal at 0x28b938c2e80>,
              deap.gp.rand101,
              <deap.gp.Terminal at 0x28b938ae4c0>]})

In [ ]:
pset.terminals.items()

dict_items([(<class 'object'>, [<deap.gp.Terminal object at 0x0000028B938A3200>, <class 'deap.gp.rand101'>, <deap.gp.Terminal object at 0x0000028B938A2C80>, <deap.gp.Terminal object at 0x0000028B93988280>])])

In [ ]:
for k in term.keys():
    print(term[k])

[<deap.gp.Terminal object at 0x0000028B938A3200>, <class 'deap.gp.rand101'>, <deap.gp.Terminal object at 0x0000028B938A2C80>, <deap.gp.Terminal object at 0x0000028B93988280>]


In [ ]:
for ret_type, terminals in pset.terminals.items():
    print(f"Return type: {ret_type}")
    for terminal in terminals:
        print(f"  Terminal: {terminal}")

Return type: <class 'object'>
  Terminal: <deap.gp.Terminal object at 0x0000028B938A3200>
  Terminal: <class 'deap.gp.rand101'>
  Terminal: <deap.gp.Terminal object at 0x0000028B938A2C80>
  Terminal: <deap.gp.Terminal object at 0x0000028B93988280>


In [ ]:
from deap import gp

# Example: Creating a PrimitiveSet and Adding Terminals
pset = gp.PrimitiveSet("MAIN", 2)  # Two arguments (ARG0, ARG1)
pset.addTerminal(42)               # Adding a constant terminal
pset.addTerminal("Hello")          # Adding another constant terminal

# Inspecting the Terminal Set
for ret_type, terminals in pset.terminals.items():
    print(f"Return type: {ret_type}")
    for terminal in terminals:
        print(f"  Terminal: {terminal}")
        print(f"    - Value: {getattr(terminal, 'value', None)}")  # The actual value
        print(f"    - Name: {getattr(terminal, 'name', None)}")    # The name of the terminal
        print(f"    - Ret Type: {terminal.ret}")                   # Return type of the terminal

Return type: <class 'object'>
  Terminal: <deap.gp.Terminal object at 0x0000028B93B7E680>
    - Value: ARG0
    - Name: ARG0
    - Ret Type: <class 'object'>
  Terminal: <deap.gp.Terminal object at 0x0000028B93B7E480>
    - Value: ARG1
    - Name: ARG1
    - Ret Type: <class 'object'>
  Terminal: <deap.gp.Terminal object at 0x0000028B93B7E2C0>
    - Value: 42
    - Name: 42
    - Ret Type: <class 'object'>
  Terminal: <deap.gp.Terminal object at 0x0000028B93B7E3C0>
    - Value: Hello
    - Name: Hello
    - Ret Type: <class 'object'>


# PSO-GP

In [6]:
import operator
import math
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from pso_utils import *
from landscapes import *

In [7]:
def gen_random(_):
    n = 500 # number of particles
    return np.random.uniform(np.zeros(2), np.ones(2), size=(n, 2))

In [8]:
n = 500

In [ ]:
pset = gp.PrimitiveSet("MAIN", 4)
# basic
pset.addPrimitive(np.add, 2)
pset.addPrimitive(np.negative, 1)
pset.addPrimitive(np.subtract, 2)
pset.addPrimitive(np.multiply, 2)

# # random
# pset.addPrimitive(gen_random, 1)

# others
# pset.addPrimitive(np.cos, 1)
# pset.addPrimitive(np.sin, 1)
# pset.addPrimitive(compute_magnitude, 1)
# pset.addPrimitive(distance, 2)


# PRIMITIVES
pset.addTerminal(-1)
pset.addTerminal(np.pi)
# ephemeral constants
pset.addEphemeralConstant()


# variables
pset.renameArguments(ARG0="swarm")
pset.renameArguments(ARG1="vel")
pset.renameArguments(ARG2='gbest')
pset.renameArguments(ARG3='pbest')

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def GPFitness(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    S = SwarmGP()
    sqerrors = ((func(x) -  target(x))**2 for x in points) # x**4 - x**3 - x**2 - x -
    return math.fsum(sqerrors) / len(points) + 0.7*len(individual),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3) # selction
toolbox.register("mate", gp.cxOnePoint) # crossover
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 100, stats=mstats,
                                   halloffame=hof, verbose=False)
    # print log
    return pop, log, hof

if __name__ == "__main__":
    pop, log, hof = main()

print(log)

TypeError: <lambda>() missing 3 required positional arguments: 'vel', 'gbest', and 'pbest'